In [21]:
"""
    Copyright (c) 2024 Idiap Research Institute, http://www.idiap.ch/
    Written by Cem Bilaloglu <cem.bilaloglu@idiap.ch>

    This file is part of diffusionVirtualFixtures.

    diffusionVirtualFixtures is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License version 3 as
    published by the Free Software Foundation.

    diffusionVirtualFixtures is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with diffusionVirtualFixtures. If not, see <http://www.gnu.org/licenses/>.
"""
"""
This notebook is used for testing the functions in point_cloud_utils.py
and diffusion_utils.py. It is also used for visualizing the results 
"""

import numpy as np
import open3d as o3d
import cv2
# from point_cloud_utils import *
from plotting_utils import *

input_img_dir = "images/"
point_cloud_dir = "point_clouds/" # for loading the point cloud

In [22]:
# Select the object and load the point cloud
# ==========================================
obj_name = "bun270"

filename = f"{point_cloud_dir}{obj_name}.ply"
pcd = o3d.io.read_point_cloud(filename)

pcd_tree = o3d.geometry.KDTreeFlann(pcd)
vertices = np.asarray(pcd.points)
print(len(vertices))

# Reset the annotation
u0 = np.ones((len(vertices),3))

31701


In [23]:
# Set the target regions on the point cloud -> Click with the cursor to add targets
#==============================================================================
fig = visualize_point_cloud(vertices,colors=u0,point_size=3,is_show_plot=True)

radius = 0.005 # radius of the brush
strip_thickness = 0.001 # thickness of the boundary strip
u0 = 0*np.ones((len(vertices),3))
target_vertices = []
boundary_vertices = []
def update_point(trace, points, selector):
# create our callback function
    c = list(trace.marker.color)
    for i in points.point_inds:
        # get the indices of the points within the radius
        k,idx, dists = pcd_tree.search_radius_vector_3d(vertices[i], radius)
        idx = np.asarray(idx).astype(int)
        for j in idx:
            # if int(j) not in boundary_vertices:
            # mark the boundary points 
            d = np.linalg.norm(vertices[j,:]-vertices[i,:])
            # mark the target points
            u0[j,:] = [0,255,0]
            c[j] = "green"
            if d > (radius-strip_thickness):
                # if it is already a target, it can't be a boundary
                if int(j) not in target_vertices: 
                    c[j] = "blue" # overwrite the color
                    u0[j,:] = [0,255,255]
                    boundary_vertices.append(j)

            target_vertices.append(j) # add after the boundary
        with fig.batch_update():
            trace.marker.color = c

target_img_name = "drawing"
fig.data[0].on_click(update_point)
fig

FigureWidget({
    'data': [{'marker': {'color': [rgb(1,1,1), rgb(1,1,1), rgb(1,1,1), ...,
                                   rgb(1,1,1), rgb(1,1,1), rgb(1,1,1)],
                         'opacity': 0.2,
                         'showscale': True,
                         'size': 3},
              'mode': 'markers',
              'name': 'Point Cloud',
              'type': 'scatter3d',
              'uid': 'a0e98189-b400-422f-9b35-310a40c42a43',
              'x': array([-0.02525, -0.02475, -0.02425, ..., -0.018  , -0.0175 , -0.017  ]),
              'y': array([0.0347151, 0.034729 , 0.0347708, ..., 0.187602 , 0.187601 , 0.187601 ]),
              'z': array([0.0434868, 0.0436137, 0.0439944, ..., 0.0190251, 0.019152 , 0.019152 ])}],
    'layout': {'scene': {'aspectmode': 'data',
                         'camera': {'center': {'x': 0, 'y': 0, 'z': 0},
                                    'eye': {'x': 0.0, 'y': 0.0, 'z': 2.0},
                                    'up': {'x': 0, 'y': 1, 'z'

In [24]:
# Save the point cloud with the target
#==============================================================================
pcd.colors = o3d.utility.Vector3dVector(u0) # r: obstacle, g: target, b: boundary
pcd.points = o3d.utility.Vector3dVector(vertices)
# filename = f"{point_cloud_dir}{obj_name}_{target_img_name}_{constraint_img_name}"
filename = f"{point_cloud_dir}{obj_name}_annotated"
o3d.io.write_point_cloud(filename+".ply", pcd)

[Open3D WARNING] Write Ply clamped color value to valid range


True